In [1]:
from pathlib import Path
import warnings
import rasterio
from rasterio.enums import Resampling
from rasterio.warp import reproject
from rasterio.mask import mask
import fiona

In [14]:


def print_grid_info(label, ds):
    print(f"{label} transform : {ds.transform}")
    print(f"{label} size      : {ds.width} × {ds.height}")
    print(f"{label} crs       : {ds.crs}")
    print(f"{label} dtype/nodata: {ds.dtypes[0]} / {ds.nodata}")
    print("-" * 60)

def align_to_reference(src_path, ref_path, out_path,
                       resampling=Resampling.nearest,
                       compress="LZW",
                       blocksize=256,
                       dst_nodata=-9999.0):
    """
    Align `src_path` to exactly match `ref_path` grid/CRS.
    1) Try tiled GeoTIFF with valid tile sizes.
    2) If GDAL still complains, fall back to striped GeoTIFF.
    """
    src_path, ref_path, out_path = map(Path, (src_path, ref_path, out_path))
    out_path.parent.mkdir(parents=True, exist_ok=True)

    with rasterio.open(ref_path) as ref, rasterio.open(src_path) as src:
        # Build a CLEAN profile (do NOT copy ref.profile to avoid stale tags)
        base_profile = {
            "driver": "GTiff",
            "width": ref.width,
            "height": ref.height,
            "count": src.count,
            "dtype": src.dtypes[0],
            "crs": ref.crs,
            "transform": ref.transform,
            "nodata": dst_nodata,
            "compress": compress,
            "interleave": "band",
            "BIGTIFF": "IF_SAFER",
        }

        if src.crs != ref.crs:
            warnings.warn(f"CRS differs (src={src.crs}, ref={ref.crs}); will reproject.")

        # Source nodata: treat NaN as nodata when dtype is float
        src_nodata = src.nodata
        if src_nodata is None and np.issubdtype(np.dtype(src.dtypes[0]), np.floating):
            src_nodata = np.nan

        def _do_reproject(profile):
            with rasterio.open(out_path, "w", **profile) as dst:
                for b in range(1, src.count + 1):
                    reproject(
                        source=rasterio.band(src, b),
                        destination=rasterio.band(dst, b),
                        src_transform=src.transform,
                        src_crs=src.crs,
                        src_nodata=src_nodata,
                        dst_transform=ref.transform,
                        dst_crs=ref.crs,
                        dst_nodata=profile["nodata"],
                        resampling=resampling,
                    )

        # 1) Try TILED
        tiled_profile = base_profile.copy()
        tiled_profile.update({
            "tiled": True,
            "blockxsize": int(blocksize),
            "blockysize": int(blocksize),  # multiples of 16 required; 256 is safe
        })
        try:
            _do_reproject(tiled_profile)
        except Exception as e:
            print(f"[warn] Tiled write failed: {e}\n         Falling back to striped GeoTIFF...")
            # 2) Fallback to STRIPED (no tiles)
            striped_profile = base_profile.copy()
            striped_profile.update({
                "tiled": False
            })
            _do_reproject(striped_profile)

    # Verify exact grid match
    with rasterio.open(out_path) as out, rasterio.open(ref_path) as ref2:
        assert out.transform == ref2.transform, "Transforms differ after alignment."
        assert (out.width, out.height) == (ref2.width, ref2.height), "Size differs after alignment."
        assert out.crs == ref2.crs, "CRS differs after alignment."
    print(f"✓ Aligned: {out_path}")

def clip_raster_to_vector(in_raster, vector_path, out_path, crop=True, compress="LZW"):
    """
    Clips `in_raster` to polygon(s) in `vector_path`. Assumes grid is already aligned
    if you plan to compute per-pixel ops with another raster on the same grid.
    """
    in_raster, vector_path, out_path = map(Path, (in_raster, vector_path, out_path))
    out_path.parent.mkdir(parents=True, exist_ok=True)

    with rasterio.open(in_raster) as src, fiona.open(vector_path, "r") as shp:
        geoms = [feat["geometry"] for feat in shp]
        out_image, out_transform = mask(src, geoms, crop=crop, nodata=src.nodata)
        out_meta = src.meta.copy()
        out_meta.update({
            "height": out_image.shape[1],
            "width": out_image.shape[2],
            "transform": out_transform,
            "compress": compress,
            "tiled": True,
            "BIGTIFF": "IF_SAFER"
        })

    with rasterio.open(out_path, "w", **out_meta) as dst:
        dst.write(out_image)
    print(f"✓ Clipped written: {out_path}")



In [15]:
# --------------------------- EXAMPLE USAGE ---------------------------

bp = r"C:\Users\bsf31\Documents\data\NL060\WFM Outputs\run_97thV2\Clip_v2\BP\Central_Coast_97th_BP_v2.tif"
cfl = r"C:\Users\bsf31\Documents\data\NL060\WFM Outputs\run_97thV2\_CFL_rasters\Clip_v2\CLF_ft\CFL_ft_CC_v2.tif"
cfl_aligned = r"C:\Users\bsf31\Documents\data\NL060\WFM Outputs\run_97thV2\_CFL_rasters\CFL_ft_CC_v2_aligned_to_BP.tif"

# 1) Align CFL to BP grid BEFORE any clipping
align_to_reference(src_path=cfl, ref_path=bp, out_path=cfl_aligned, resampling=Resampling.nearest)




[warn] Tiled write failed: CFL_ft_CC_v2_aligned_to_BP.tif: TIFFReadDirectory:Cannot handle zero number of tiles
         Falling back to striped GeoTIFF...


CPLE_AppDefinedError: CFL_ft_CC_v2_aligned_to_BP.tif: TIFFReadDirectory:Cannot handle zero number of tiles

In [ ]:
#2) (Optional) Clip both rasters using the *same* AOI *after* they share the same grid
aoi = r"C:\Users\bsf31\Documents\data\NL060\fire_scar_training_regions.gpkg"  # or .shp
bp_clip  = r"C:\...\IH_Output\CC\BP_clip.tif"
cfl_clip = r"C:\...\IH_Output\CC\CFL_clip.tif"
clip_raster_to_vector(bp,  aoi, bp_clip)
clip_raster_to_vector(cfl_aligned, aoi, cfl_clip)